In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
HIDDEN_UNITS = 256
N_CLASSES = 10
LEARNING_RATE = 1e-2
TRAINING_ITER = 1000
BATCH_SIZE = 100

def BiRNN(x, weights, biases):
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, 28])
    x = tf.split(x, 28)
    
    lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(num_units=HIDDEN_UNITS)
    lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(num_units=HIDDEN_UNITS)
    
    outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
    
    return tf.matmul(outputs[-1], weights) + biases


x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_2d = tf.reshape(x, [-1, 28, 28])

weights = {'output': tf.Variable(tf.random_normal([2 * HIDDEN_UNITS, N_CLASSES]))}
biases = {'output': tf.Variable(tf.zeros([N_CLASSES]) + 0.1)}

predictions = BiRNN(x_2d, weights['output'], biases['output'])

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=predictions))
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(y, 1), tf.arg_max(predictions, 1)), tf.float32))


init = tf.global_variables_initializer()


In [5]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(TRAINING_ITER):
        batch_xs, batch_ys = mnist.train.next_batch(BATCH_SIZE)
        sess.run(train_step, feed_dict = {x: batch_xs, y: batch_ys})
        if i % 50 == 0:
            print sess.run(accuracy, feed_dict = {x: batch_xs, y: batch_ys})
            print sess.run(accuracy, feed_dict = {x: mnist.validation.images, y: mnist.validation.labels})

0.15
0.73
0.84
0.92
0.95
0.91
0.95
0.98
0.99
0.97
0.97
0.99
0.99
0.98
0.99
0.99


KeyboardInterrupt: 